In [1]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

class TwitterClient(object): 
	''' 
	Generic Twitter Class for sentiment analysis. 
	'''
	def __init__(self): 
		''' 
		Class constructor or initialization method. 
		'''
		# keys and tokens from the Twitter Dev Console 
		consumer_key = '9RkYhudUQkKKHEuCwnU1WmKGt'
		consumer_secret = 'bkRl7MNNSzi2Hvg1bJUb3weakYw9Rlf9v35thqPLQ3YcViWCjA'
		access_token = '357276346-oIxMqGkzb82Vz7uE6cEKCX5l8HUKNwPmTVKAADNR'
		access_token_secret = 'fsph2QGPKvL8dDRRu97IgICqCdGNnf44cIkTXS2ctaaH2'

		# attempt authentication 
		try: 
			# create OAuthHandler object 
			self.auth = OAuthHandler(consumer_key, consumer_secret) 
			# set access token and secret 
			self.auth.set_access_token(access_token, access_token_secret) 
			# create tweepy API object to fetch tweets 
			self.api = tweepy.API(self.auth) 
		except: 
			print("Error: Authentication Failed") 

	def clean_tweet(self, tweet): 
		''' 
		Utility function to clean tweet text by removing links, special characters 
		using simple regex statements. 
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

	def get_tweet_sentiment(self, tweet): 
		''' 
		Utility function to classify sentiment of passed tweet 
		using textblob's sentiment method 
		'''
		# create TextBlob object of passed tweet text 
		analysis = TextBlob(self.clean_tweet(tweet)) 
		# set sentiment 
		if analysis.sentiment.polarity > 0: 
			return 'positive'
		elif analysis.sentiment.polarity == 0: 
			return 'neutral'
		else: 
			return 'negative'
        
	def get_tweet_sentiment_polarity(self, tweet): 
		''' 
		Utility function to classify sentiment of passed tweet 
		using textblob's sentiment method 
		'''
		# create TextBlob object of passed tweet text 
		analysis = TextBlob(self.clean_tweet(tweet)) 
		# set sentiment 
		if analysis.sentiment.polarity > 0: 
			return analysis.sentiment.polarity
		elif analysis.sentiment.polarity == 0: 
			return 'neutral'
		else: 
			return analysis.sentiment.polarity
        
	def get_tweets(self, query, count = 100): 
		''' 
		Main function to fetch tweets and parse them. 
		'''
		# empty list to store parsed tweets 
		tweets = [] 

		try: 
			# call twitter api to fetch tweets 
			fetched_tweets = self.api.search(q = query, count = count) 

			# parsing tweets one by one 
			for tweet in fetched_tweets: 
				# empty dictionary to store required params of a tweet 
				parsed_tweet = {} 

				# saving text of tweet 
				parsed_tweet['text'] = tweet.text 
				# saving sentiment of tweet 
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
				# saving sentiment of tweet 
				parsed_tweet['polarity'] = self.get_tweet_sentiment_polarity(tweet.text)


				# appending parsed tweet to tweets list 
				if tweet.retweet_count > 0: 
					# if tweet has retweets, ensure that it is appended only once 
					if parsed_tweet not in tweets: 
						tweets.append(parsed_tweet) 
				else: 
					tweets.append(parsed_tweet) 

			# return parsed tweets 
			return tweets 

		except tweepy.TweepError as e: 
			# print error (if any) 
			print("Error : " + str(e)) 

def main(): 
	# creating object of TwitterClient Class 
	api = TwitterClient() 
	# calling function to get tweets 
	tweets = api.get_tweets(query = 'Donald Trump', count = 40000) 

	# picking positive tweets from tweets 
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
	# percentage of positive tweets 
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
	# picking negative tweets from tweets 
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
	# percentage of negative tweets 
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
	# percentage of neutral tweets 
	print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets))) 

	# printing first 5 positive tweets 
	print("\n\nPositive tweets:") 
	for tweet in ptweets[:10]: 
		print(tweet['text'],tweet['polarity']) 

	# printing first 5 negative tweets 
	print("\n\nNegative tweets:") 
	for tweet in ntweets[:10]: 
		print(tweet['text']) 

if __name__ == "__main__": 
	# calling main function 
	main() 



Positive tweets percentage: 25.35211267605634 %
Negative tweets percentage: 15.492957746478874 %
Neutral tweets percentage: 59.15492957746479 %


Positive tweets:
RT @ChrisJZullo: I support @JoeBiden but let’s be clear

If Bernie wins, I’ll support Bernie
If Warren wins, I’ll support Warren
If Pete wi… 0.2333333333333333
RT @PoeticJ59274248: Does the Constitution say how many impeachments can happen at the same time? There are so many deserving candidates.… 0.4
RT @cjdtwit: 📢 BREAKING!

Trump Signs Landmark Trade Deal with China to Fix 'Wrongs of the Past'!

Our president is incredible! There is NO… 0.325
RT @PalmerReport: Some of these democratic candidates are so much better than others. But even if my least favorite candidate gets the demo… 0.2333333333333333
RT @JonathanTurley: The question is whether Democrats want to enthrone a new monarch after deposing the old one. But if Democrats are only… 0.07878787878787878
RT @AFunnyValentine: THE TAKEAWAY about the
FINAL DEBATE before Fir